In [1]:
## Packages lists:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.optimize import minimize

## Importing all the dataframes from data.ipynb
%run data.ipynb

In [2]:
df_m = monthly_return.pct_change().fillna(0)
df_m.index = pd.to_datetime(df_m.index)
df_m = df_m.iloc[:,:15] ## small sample to test quickly the code
df_m.head()

,SCHLUMBERGER,ALUAR,BBVA BANCO FRANCES,TERNIUM ARGENTINA SOCIEDAD ANONIMA,FLUGHAFEN WIEN,ERSTE GROUP BANK,OMV,VERBUND,WIENERBERGER,VIENNA INSURANCE GROUP A,VOESTALPINE,MAYR-MELNHOF KARTON,IMMOFINANZ,ADBRI,ARISTOCRAT LEISURE
2000-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2000-02-29,0.213043,0.050002,0.178923,0.039543,-0.015488,0.029818,-0.181937,0.076230,-0.073916,-0.032720,-0.045831,0.023369,-0.009660,-0.316150,-0.285561
2000-03-31,0.035756,-0.023552,-0.105072,-0.002113,0.031138,0.028238,-0.023027,-0.081528,0.112531,-0.005999,0.009498,-0.083787,-0.001781,-0.011631,-0.148047
2000-04-30,0.000814,-0.016173,-0.161842,-0.071188,-0.134598,-0.043483,0.076523,-0.106429,0.041304,-0.021831,-0.079973,-0.060966,-0.046843,-0.117176,-0.042805
2000-05-31,-0.039184,-0.016727,0.005304,-0.341421,0.085088,0.017996,0.064640,0.074843,0.075991,0.049232,-0.034112,0.030421,0.048088,-0.152639,0.154330


In [3]:
def roll_min_var_opt(df, start, end):
    stocks = df.columns
    year_weight = end.year + 1
    weights_df = pd.DataFrame(index=stocks)
    
    ## LATER Don't forget to remove the dropstocks

    for i in range(17):
        ## Resample from start to end
        sample_m = df[df.index.isin(pd.date_range(start, end))]
        tau = len(sample_m)

        ## Compute expected returns
        mu_hat = pd.DataFrame(sample_m.mean(axis=0)).T
    
        ## Excess returns
        excess_returns = sample_m.subtract(mu_hat.values.squeeze(), axis=1)
    
        ## Cov Mat
        covmat = 1/tau * excess_returns.T @ excess_returns
    
    
        # Define objective function (portfolio variance)
        def portfolio_variance(weights, covmat):
            return np.dot(weights.T, np.dot(covmat, weights))
    
        # Define constraint (sum of weights equals 1)
        def constraint(weights):
            return np.sum(weights) - 1
    
        n_assets = len(covmat)
    
        # Initial guess for weights
        initial_weights = np.ones(n_assets) / n_assets
        
        # Define bounds for weights (0 to 1) long-only portfolio
        bounds = [(0, None)] * n_assets
    
        result = minimize(portfolio_variance, initial_weights, args=(covmat,), constraints={'type': 'eq', 'fun': constraint}, bounds=bounds)
    
        optimal_weights = result.x

        weights_df[year_weight] = optimal_weights

        start = dt.datetime(start.year + 1, 1, 1)
        end = dt.datetime(end.year + 1, 1, 1)
        year_weight += 1

    return weights_df

In [4]:
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2005, 12, 31)
weights = roll_min_var_opt(df_m, start, end)

weights_melted = weights.reset_index().melt(id_vars='index', var_name='stock', value_name='weight')
weights_melted.head()


,index,stock,weight
0,SCHLUMBERGER,2006,5.747065e-02
1,ALUAR,2006,6.281954e-02
2,BBVA BANCO FRANCES,2006,1.246832e-17
3,TERNIUM ARGENTINA SOCIEDAD ANONIMA,2006,0.000000e+00
4,FLUGHAFEN WIEN,2006,0.000000e+00
